# 1 : Librairies Python

In [1]:
# pip install pandas sqlalchemy psycopg2-binary schedule

In [2]:
# Librairies et options :

import pandas as pd
import random
import datetime
import requests
import json
import numpy as np
import tqdm
import time
import pytz

from sqlalchemy import create_engine
import psycopg2
from sqlalchemy.types import Integer, BigInteger, JSON

# 2 : Récupération de la TS la plus récente

In [3]:
# Paramètres de connexion à la base de données PostgreSQL en local :
host = "localhost"
database = "Test_LPO"
user = "postgres"
password = "15072022"

In [4]:
# Connexion à la base de données
conn = psycopg2.connect(dbname = database, user = user, password = password , host = host)

# Création d'un curseur : permet d'exécuter des commandes SQL sur la base de données.
cur = conn.cursor()

# Exécution d'une requête SQL pour sélectionner les données de ma_table
cur.execute("SELECT ts FROM historiquemeteo ORDER BY ts DESC LIMIT 1")

# Récupération des données dans une liste de tuples
data = cur.fetchall()

# Création d'un DataFrame à partir des données
df = pd.DataFrame(data, columns=[desc[0] for desc in cur.description])

# Fermeture du curseur et de la connexion
cur.close()
conn.close()

# Récupération de la ts la plus récente de la BDD :
LastLoadTimestamp = df.values[0][0]
LastLoadTimestamp

1712872800

# 3 : Ouverture de l'API

In [5]:
# Informations : https://weatherlink.github.io/v2-api/

APIKey = 'zjbvtfdvjslxklxclzscfu6isqpqfxbg'
APISecret = 'mih0udjsuetwaatpwzrqzeglljmy889b'
StationID = 122495

In [6]:
# Date d'aujourd'hui à minuit :
Today = datetime.date.today()
TodayMidnight = datetime.datetime.combine(Today, datetime.time.min)

# Dernier enregistrement dans la BDD :
LastLoadTimestamp = df.values[0][0]

# Convertir la date :
TodayMidnightTimestamp = int(TodayMidnight.timestamp())

print("Le timestamp correspondant à la date du {} est :".format(TodayMidnight), TodayMidnightTimestamp)

Le timestamp correspondant à la date du 2024-04-13 00:00:00 est : 1712959200


In [7]:
# DataFrame historiques :
dfNew = pd.DataFrame()

# Nb de jours à récupérer :
NbJours = int((TodayMidnightTimestamp - LastLoadTimestamp) / 86400)

for i in tqdm.tqdm(range(NbJours)):
    StartTime = LastLoadTimestamp + i * 86400
    EndTime = StartTime + 86400
    
    # Lien de la request : 
    link = 'https://api.weatherlink.com/v2/historic/{}?api-key={}&start-timestamp={}&end-timestamp={}'.format(StationID, APIKey, StartTime, EndTime)
    headers = {'X-Api-Secret' : APISecret}

    # Requête :
    r = requests.get(link, headers=headers)

    # Si la requête a réussi :
    if r.status_code == 200:
        # Lecture de la request en json :
        data = r.json()

        # Transformation en DF : 
        dfJour = pd.DataFrame(data)
        dfJour = dfJour[['station_id','sensors']]

        # Récupération des valeurs se trouvant dans sensors :
        dfSensors = pd.json_normalize(data['sensors'][0]['data'])
    
        # Récupération des json sur une colonne :
        dfJour = pd.DataFrame({'station_id': data['station_id'], 'infos_json' : data['sensors'][0]['data']})
    
        # Convertir les objets JSON en chaînes de caractères JSON :
        dfJour['infos_json'] = dfJour['infos_json'].apply(json.dumps)

        # Concat des données :
        dfJour = pd.concat([dfJour, dfSensors], axis=1)
        
        # Concaténation des données :
        dfNew = pd.concat([dfNew, dfJour], ignore_index=True)
    else:
        print("La requête {} a échoué avec le code d'erreur {}".format(link, r.status_code))

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


In [10]:
# Data Frame Finale :
dfNew

,station_id,infos_json,ts,tz_offset,arch_int,rev_type,temp_out,temp_out_hi,temp_out_lo,temp_in,...,wind_run,deg_days_heat,deg_days_cool,solar_energy,uv_dose,thw_index,thsw_index,wet_bulb,night_cloud_cover,iss_reception
0,122495,"{""ts"": 1712873100, ""tz_offset"": 7200, ""arch_in...",1712873100,7200,300,2,50.1,50.1,50.0,49.1,...,0.333333,0.051736,0.0,0.0,0.0,48.565760,44.539604,45.446,0.0,None
1,122495,"{""ts"": 1712873400, ""tz_offset"": 7200, ""arch_in...",1712873400,7200,300,2,50.2,50.2,50.1,49.0,...,0.333333,0.051389,0.0,0.0,0.0,48.661280,44.635120,45.500,0.0,None
2,122495,"{""ts"": 1712873700, ""tz_offset"": 7200, ""arch_in...",1712873700,7200,300,2,50.2,50.3,50.2,49.0,...,0.333333,0.051389,0.0,0.0,0.0,48.625282,44.599120,45.320,0.0,None
3,122495,"{""ts"": 1712874000, ""tz_offset"": 7200, ""arch_in...",1712874000,7200,300,2,50.4,50.4,50.2,49.0,...,0.333333,0.050694,0.0,0.0,0.0,48.820312,44.794155,45.500,0.0,None
4,122495,"{""ts"": 1712874300, ""tz_offset"": 7200, ""arch_in...",1712874300,7200,300,2,50.7,50.7,50.4,49.1,...,0.333333,0.049653,0.0,0.0,0.0,49.112860,45.086700,45.824,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,122495,"{""ts"": 1712958000, ""tz_offset"": 7200, ""arch_in...",1712958000,7200,300,2,59.5,59.6,59.5,59.4,...,0.416667,0.019097,0.0,0.0,0.0,58.376300,54.718132,53.240,0.0,None
284,122495,"{""ts"": 1712958300, ""tz_offset"": 7200, ""arch_in...",1712958300,7200,300,2,59.4,59.5,59.4,59.2,...,0.333333,0.019444,0.0,0.0,0.0,58.396000,54.737830,53.240,0.0,None
285,122495,"{""ts"": 1712958600, ""tz_offset"": 7200, ""arch_in...",1712958600,7200,300,2,59.2,59.4,59.2,59.0,...,0.333333,0.020139,0.0,0.0,0.0,58.188000,54.529830,53.060,0.0,None
286,122495,"{""ts"": 1712958900, ""tz_offset"": 7200, ""arch_in...",1712958900,7200,300,2,59.0,59.2,59.0,58.8,...,0.250000,0.020833,0.0,0.0,0.0,57.980000,54.321830,52.880,0.0,None


# 3 : Transfert sur PostgreSQL

In [9]:
# Début du chronomètre :
start_time = time.time()

# Création de la chaîne de connexion PostgreSQL :
conn_str = f"postgresql://{user}:{password}@{host}/{database}"

# Création de la connexion à la base de données PostgreSQL :
engine = create_engine(conn_str)

# Définir les types de données pour chaque colonne :
dtype = {'station_id': Integer(),
         'ts': BigInteger(),
         'infos_json': JSON}

# Ajouter le DataFrame dans la base de données PostgreSQL :
dfNew.to_sql('historiquemeteo', engine, if_exists = 'append', index=False, dtype=dtype)

# Fermeture de la connexion :
engine.dispose()

# Fin du chronomètre :
end_time = time.time()

# Calcul de la durée d'exécution :
execution_time = end_time - start_time

# Affichage de la durée d'exécution :
print("Durée d'exécution :", round(execution_time/60,2), "minutes")

Durée d'exécution : 0.01 minutes
